# OPG : LPA Data Pre-processing and Cleaning tool

==============================================================================

 OPG Demand Forecast modelling for LPA
 
 @author: Leila Yousefi
 
 MoJ Modelling Hub
 
============================================================================== 

## Setup

Before you can run this project, you need to install some Python packages using the terminal:


### create and activate  a virtual environment
1. cd OPG
2. python3 -m venv venv
3. source venv/bin/activate

### install the python packages required
4. pip install --upgrade pip
5. pip install -r requirements.txt

### Updating your branch with main
When working on your models it is likely that your branch will get out of date with the main branch. To update you branch with the latest changes from main open a terminal and run the following:

Check your working tree, commit/push any changes if required

git status
Switch to the main branch and collect the latest changes, if any

git switch main
git fetch
git pull
Switch back to your branch and merge in the changes from main

git switch <your initial>/model-a-development
git merge main -m "update branch with main"

# Installing the required packages:

In [ ]:
# Un-comment and Run the below code if there is an error with packages installation:

#!pip install arrow_pd_parser
#!pip install pydbtools
#!pip install arrow_pd_parser
#!pip install pydbtools
#!pip install xlsxwriter


##You can add lines to install the required packages

# Importing the required packages:

In [ ]:

import sys
print(sys.path)

import os.path
from os import path
os.getcwd()

import pandas as pd
import numpy as np
#import awswrangler as wr
#import statsmodels.api as sm
#import tensorflow as tf
import boto3
import getpass
import pytz
#import openpyxl
#import matplotlib
import csv
from arrow_pd_parser import reader, writer
import shutil
import pydbtools as pydb
import datetime as dt
from datetime import timedelta
from datetime import datetime
from datetime import date

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt

from io import StringIO

#from tensorflow import keras
import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 10000
from matplotlib import rc
import statsmodels.api as sm

# consistent plot size wherever not specifiied
from pylab import rcParams
mpl.rcParams['figure.figsize'] = (15,8)
mpl.rcParams['axes.grid'] = False
rcParams['xtick.labelsize'] = 14
rcParams['ytick.labelsize'] = 14
rcParams['axes.labelsize'] = 14

%matplotlib inline
%config InlineBackend.figure_format='retina'

import xlsxwriter
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import rc
import statsmodels.api as sm

# consistent plot size wherever not specifiied
from pylab import rcParams
mpl.rcParams['figure.figsize'] = (15,8)
mpl.rcParams['axes.grid'] = False
rcParams['xtick.labelsize'] = 14
rcParams['ytick.labelsize'] = 14
rcParams['axes.labelsize'] = 14


# LPA Data Import from Dom1

This was used previously before transfering data to AWS

In [ ]:
# ## Enter the corresponding bucket name
# bucketName = "alpha-opg-analytical"

# ##For Automation import getpass
# #bucketName = getpass.getpass()


# ## Select the folder
# folderPath = "sirius_data_cuts_3"


# ## Set the folder in which the final output will be uploade to in S3
# #output_path = f"s3://alpha-opg-analytical/" + folderPath + "/"

# ## Then create a new excel file and copy the previous record from the S3 buckets and add the newly copied raws
# ## Finaly convert the excel file to csv and upload it in the following path:
# ## s3://alpha-opg-analytical/sirius_data_cuts_3/


# ## Explore the s3 bucket path
# path_s3 = f"s3://{bucketName}/{folderPath}/*.csv"
# print ([path_s3])


# ## Listing CSV Files in an S3 Bucket Folder: 
# ### To list all CSV files in a specific folder within an S3 bucket, we can use the AWS CLI or the boto3 Python library. 
# ###list all files in a specific folder within an S3 bucket Using AWS CLI:
# #aws s3 ls s3://your-bucket-name/your-folder-name/ --recursive

# ### lists all CSV files in a specific folder within an S3 bucket using boto3:
# def list_csv_files(bucketName, folderPath):
#     s3 = boto3.resource('s3')
#     bucket = s3.Bucket(bucketName)
#     for obj in bucket.objects.filter(Prefix=folderPath):
#         if obj.key.endswith('.csv'):
#             print(obj.key)


# ## list all csv in the current folder in s3 bucket:
# ### Getting the List of CSV Files
# csv_files = list_csv_files(bucketName, folderPath)
# print([csv_files])


# ## Check if the path exists:
# s3 = boto3.resource('s3')
# bucket = s3.Bucket(bucketName)

# def IsObjectExists(path):
#     for object_summary in bucket.objects.filter(Prefix=path):
#         return True
#     return False

# for fileName in [csv_files]:
#     if(IsObjectExists(f"{folderPath}/{fileName}")):
#         print("Path for the actual LPA data exists")
#     else:
#         print("Path for the actual LPA data doesn't exists")
        

# S3 Bucket Data Extraction for LPA Data (actuals)

These will be used when extracting the raw data from the S3

In [ ]:
# Download and upload the LPA actual data into the S3 bucket

## Enter the corresponding S3 bucket name
bucketName = "alpha-opg-analytical"

##For Automation import getpass
#bucketName = getpass.getpass()


## Select the corresponding folder includes new LPA data in S3 bucket:
folderPath = "sirius_data_cuts_3"


## Set the folder in which the final output will be uploade to in S3
#output_path = f"s3://alpha-opg-analytical/" + folderPath + "/"

## Then create a new excel file and copy the previous record from the S3 buckets and add the newly copied raws
## Finaly convert the excel file to csv and upload it in the following path:
## s3://alpha-opg-analytical/sirius_data_cuts_3/

# Initialize the S3 client
s3_client = boto3.client('s3')

# Specify your bucket name and folder (prefix)
bucket_name = bucketName
folder_prefix = 'sirius_data_cuts_3/'

# List objects in the specified folder
response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=folder_prefix)

# Extract the keys (file names) from the response
file_keys = [obj['Key'] for obj in response.get('Contents', [])]

# Filter out None values (if any)
non_none_file_keys = [key for key in file_keys if key is not None]
#print(non_none_file_keys)

# Remove folder prefix from file keys
file_names = [os.path.basename(key) for key in non_none_file_keys]
#print(file_names)

csv_extension = '.csv'
filtered_file_names = [fn for fn in file_names if fn.lower().endswith(csv_extension)]

print(filtered_file_names)

# ## Explore the s3 bucket path
# path_s3 = f"s3://{bucketName}/{folderPath}/"
# print ([path_s3])

# ## Check if the path exists:
# s3 = boto3.resource('s3')
# bucket = s3.Bucket(bucketName)

# def IsObjectExists(path):
#     for object_summary in bucket.objects.filter(Prefix=path):
#         return True
#     return False

# if(IsObjectExists(path_s3)):
#     print("Path for the actual data exists")
# else:
#     print("Path for the actual data doesn't exists")


## Listing CSV Files in an S3 Bucket Folder: 
### To list all CSV files in a specific folder within an S3 bucket, we can use the AWS CLI or the boto3 Python library. 
###list all files in a specific folder within an S3 bucket Using AWS CLI:
#aws s3 ls s3://your-bucket-name/your-folder-name/ --recursive

# ### lists all CSV files in a specific folder within an S3 bucket using boto3:
# def list_csv_files(bucketName, folderPath):
#     s3 = boto3.resource('s3')
#     bucket = s3.Bucket(bucketName)
#     for obj in bucket.objects.filter(Prefix=folderPath):
#         if obj.key.endswith('.csv'):
#             print(obj.key)


# ## list all csv in the current folder in s3 bucket:
# ### Getting the List of CSV Files
# csv_files = list_csv_files(bucketName, folderPath)
# print([csv_files])


# ## Opening CSV Files Based on Selected Column and Condition: 
# def process_csv_file(file_path, selected_column, condition):
#     df = pd.read_csv(file_path)
#     filtered_df = df[df[selected_column] == condition]
#     # Do further processing with the filtered data

# ## pre-process the csv file to only show the required columns (important variable for the BAU):
# process_csv_file('path/to/your-csv-file.csv', 'column_name', 'desired_value')


# # ## Enter your file name
# # fileName = "d24_2.csv"


# Query the warehouse tables directly from Python/R


    """
    with events as (
        select *
        from "dim_guardianship_dev_dbt"."fct_case_receipts"
        where extract_type = 'latest_extract'
            and receipt_date >= date_parse('01-01-2008', '%d-%m-%Y')
    ),
    dates as (
        select *
        from "common_lookup_dev_dbt"."dim_date"
    ),
    donors as (
        select *
        from "dim_guardianship_dev_dbt"."dim_donors"
    ),
    cases as (
        select *
        from "dim_guardianship_dev_dbt"."dim_cases"
    ),
    attributes as (
        select dates.calendar_year as receipt_year,
            events.receipt_date,
            cases.case_id,
            cases.case_type,
            cases.case_subtype,
            cases.case_status,
            cases.donor_age_at_receipt,
            donors.gender,
            donors.region_name,
            events.extract_date
        from events
            left join dates on events.receipt_date = dates.date_name
            left join cases on events.extract_case_id = ces.extract_case_id
            left join donors on events.extract_donor_id = donors.extract_donor_id
    )
    select *
    from attributes
    """
    

# Reading in Data

This extracts a list of Power of Attorney receipts with the following columns: ['receiptdate', 'uid', 'type', 'casesubtype', 'status', 'donor_postcode', 'donor_gender', 'age'].

In [ ]:
# read_csv_files function for Reading in CSV Files in an S3 Bucket Folder

def read_csv_files(bucket_name, file_names, selected_columns):
    """
        This function is written to read in the data from all of CSV files in the corresponding directory in the S3 bucket
        by using input variables:
        the S3 bucket name,
        file_names 
        and the selected_columns 
        The output are the CSV files in the list of dataframes: dfs 
    """
    dfs = {}  # Dictionary to store DataFrames

    # Initialize the S3 client
    s3_client = boto3.client('s3')

    for file_name in file_names:
        s3_path = f's3://{bucket_name}/{file_name}'
        try:
            # Read the CSV data into a Pandas DataFrame
            csv_obj = s3_client.get_object(Bucket=bucket_name, Key=f'{folderPath}/{file_name}')
            csv_string = csv_obj['Body'].read().decode('utf-8')
            df = pd.read_csv(StringIO(csv_string))

            # Select specific columns
            df_selected = df[selected_columns]
            dfs[file_name] = df_selected
        except Exception as e:
            print(f"Error reading {file_name}: {e}")

    return dfs


#bucket_name = bucketName
#file_names = ['file1.csv', 'file2.csv']  # Replace with your actual file names

## Filter the required variables from the datafarame:
selected_columns = ["receiptdate","cases_glueexporteddate","uid","type","casesubtype","status","donor_dob","donor_postcode","donor_gender"]  # Replace with desired column names

## The read_csv_files function for Reading in CSV Files in an S3 Bucket Folder:
dataframes = read_csv_files(bucket_name, filtered_file_names, selected_columns)

## Access individual DataFrames by file name
for file_name, df_selected in dataframes.items():
    print(f"DataFrame for {file_name}:")
    print(df_selected.head())
    
## Concatenating DataFrames: 
### After reading all CSV files, you can concatenate the DataFrames using pd.concat:
combined_df = pd.concat(dataframes, ignore_index=True)
print(combined_df)

## Writing Back to S3: Finally, write the combined DataFrame back to S3:
#combined_data_encoded = combined_df.to_csv(None, index=False).encode('utf-8')
#combined_file_name = 'combined_data.csv'  # Choose a suitable file name
#s3_client.put_object(Body=combined_data_encoded, Bucket=bucket_name, Key=combined_file_name

## Identify the type of data set and pre-processing: 
## Import, manipulate, and clean the data and impute missing values

## Column renaming:
#df1.rename(columns={'old_col1': 'common_col1', 'old_col2': 'common_col2'}, inplace=True)

## Handling Data Mismatch:
###Be cautious when combining data with different structures. If a column has incompatible data types (e.g., mixing strings and numbers), you may need to convert or handle them appropriately.
#combined_df['numeric_col'] = pd.to_numeric(combined_df['numeric_col'], errors='coerce')

## Aggregating Data:
###If the DataFrames have different structures, consider aggregating them based on a common identifier (e.g., date or unique ID).
#combined_df = df1.groupby('product_id').sum()  # Aggregate by product ID

## merge DataFrames based on a common identifier:
#merged_df = pd.merge(df1, df2, on='ID', how='inner')
#print(merged_df)

In [ ]:
#     ## Select Date
#     start_date = '2018-06-01' # start date for the train set
#     start_prediction ='2023-02-01' # The end date for the train set
#     end_prediction ='2024-02-01' # test / Validation set



## Import the dataset and read in the actual data
#df = wr.s3.read_csv([path1_s3], sep = ',', parse_dates=True) #import divorce data
#read data
#def parser(s):
#    return datetime.strptime(s, '%Y-%m-%d')
#df = wr.s3.read_csv([path1_s3], parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
## iterating the columns
#for col in df.columns:
#    print(col)


#lpa=LPA_data[["receiptdate","cases_glueexporteddate","uid","type","casesubtype","status","donor_dob","donor_postcode","donor_gender"]]

# Automating the input dates to forecast LPAs


In [ ]:
# Set the date you want to extract data based on the latest date extrated LPA data

## Grab part of filename
fist_CSV_fileName = filtered_file_names[1]
snapshot_end = fist_CSV_fileName.split('opg-analytical_cases_P')[1].lstrip().split('_S')[0]
#snapshot_end

#snapshot_end = final_df.values[7].astype(str)[7]

## Automating the input dates to forecast
p = getpass.getpass(prompt='Do you want to change the starting date for forecasting? (Choose Yes=Y OR No=N)')
 
if (p.lower() == 'n') | (p.lower() == ''): #defult start date
    snapshot_start = '2006-12-31'
    print('You have not choosen to change the date to: ' + snapshot_start)
    ## The first date to be considered:
else:    
    ## Select Date
    print('You have choosen to change the date.')
    snapshot_start = input('Enter the period_start date (for training): e.g., "2006-12-31"')
    print('snapshot_start: ' + snapshot_start)

# Data pre-processing and cleaning - data engineering

## Meta data and Variable selection and Data Cleaning for the LPA data in Data Warehouse:

Goal: to work out how many people applied for lpa and recieved the power of atthorney and how many applications in a year/month/week by age group since 2007? 

### ages over 19 years old

#### Unique case reference for each donor = [donor_dob + donor_postcode + donor_gender]

##### Sort by the unique id and count how many application

###### and then dermine Whether the application type [casesubtype] is hw=health and welfare or pfa=property and finance

###### how many certificate provider (cp) for each lpa application?

###### Location based data and geographical data for the donor can be used to identify the financial situation and wherether they are located in England or Wales


In [ ]:

# Filter the records:
df_filtered = combined_df

## Convert the receipt date to date format 
df_filtered['receiptdate'] = pd.to_datetime(df_filtered['receiptdate'], errors = 'coerce').dt.date

## Filter records between the selected dates
df_filtered = df_filtered.loc[(df_filtered['receiptdate'] > pd.to_datetime(snapshot_start))]
df_filtered = df_filtered.loc[(df_filtered['receiptdate'] < pd.to_datetime(snapshot_end))]

## Filter the dataframe to select only lpa type records
df_filtered = df_filtered.loc[(df_filtered['type'] == 'lpa')]

# Create a dataframe of the selected columns
## Select the appropriate variable to be forecasted
df = df_filtered[["receiptdate","uid","casesubtype","status","donor_dob","donor_postcode","donor_gender"]]

## Remove Null values and records
lpa_df = df.dropna()

# Extract age by subtracting 'receiptdate' and 'donor_dob'
lpa_df['age'] = pd.to_datetime(lpa_df['receiptdate'], errors = 'coerce').dt.year - pd.to_datetime(lpa_df['donor_dob'], errors = 'coerce').dt.year
#lpa_df['age'] = relativedelta(date, dob).years

# Convert the donor_dob column to a datatime format
lpa_df['donor_dob'] = pd.to_datetime(lpa_df['donor_dob'], errors = 'coerce').dt.date

# Convert the ‘receiptdate’ column to datetime format for proper plotting.
# Convert 'receiptdate' to datetime format 
lpa_df['receiptdate'] = pd.to_datetime(lpa_df['receiptdate'], errors='coerce')

# Extract year from 'receiptdate'
lpa_df['year'] = lpa_df['receiptdate'].dt.year

## Set index
#df['receiptdate'] = pd.to_datetime(df['receiptdate'])

#df = df.set_index('receiptdate').asfreq('D')

####df['receiptdate'] = df.set_index('receiptdate',inplace=True)

#df.index = df.index.to_period('D')
                            
print(lpa_df.head())
print(lpa_df.tail())


#lpa_df['age'] = pd.to_datetime(df['receiptdate'], errors = 'coerce').dt.date - pd.to_datetime(df['donor_dob'], errors = 'coerce').dt.date
#lpa_df['receiptdate'] = pd.to_datetime(lpa_df['receiptdate']).dt.date#.apply(lambda x: x.strftime('%Y-%m-%d'))
#print(lpa_df)#['receiptdate']
#lpa_df

#print(lpa_df['age'])

## infer the frequency of the data:
###lpa_df = df

#lpa_df = df.asfreq(pd.infer_freq(df.index))

#lpa_df = lpa_df[start_date:end_date]

#start_date_years = datetime.strptime(start_date, 
#                                     '%Y-%m-%d') + relativedelta(years = 0)
#print(start_date_years)

#start_date_formatted = start_date_years.date()

# Visualisation of the time series
## Virtualisation of the LPA Data:

# Plot 'age' against 'receiptdate'

## --------------------------------------------------------------------------------  ##
# Create a scatter plot with ‘receiptdate’ as the x-axis and ‘age’ as the y-axis.
# Display the plot with appropriate labels and a grid.

plt.figure(figsize=(20, 10))
plt.scatter(lpa_df['receiptdate'], lpa_df['age'], alpha=0.5)
plt.title('Age vs Receipt Date')
plt.xlabel('Receipt Date')
plt.ylabel('Age')
plt.grid(True)
plt.show()

## --------------------------------------------------------------------------------  ##
# Create a histogram of the 'age' column

# This code will produce a histogram that displays the frequency distribution of ages in your dataset. 
# The bins parameter determines the number of bins used in the histogram, and you can adjust this number
# to change the granularity of your histogram.

plt.figure(figsize=(20, 10))
plt.hist(lpa_df['age'], bins=20, alpha=0.7, color='blue')
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

## --------------------------------------------------------------------------------  ##
# Create a line chart of age against receipt date
# Sort the DataFrame by 'receiptdate' to ensure the line chart is ordered
lpa_df.sort_values('receiptdate', inplace=True)

# Plot 'age' against 'receiptdate' using a line chart
plt.figure(figsize=(20, 10))
plt.plot(lpa_df['receiptdate'], lpa_df['age'], marker='o', linestyle='-', color='blue')
plt.title('Age vs Receipt Date')
plt.xlabel('Receipt Date')
plt.ylabel('Age')
plt.grid(True)

## --------------------------------------------------------------------------------  ##
# Produce a line chart that displays the average age of individuals for each year based on the receipt dates in your dataset.
# The data points are connected with a line, which helps in identifying any trends or patterns over the years.

# Group the data by year and calculate the average age for each year
age_by_year = lpa_df.groupby('year')['age'].mean().reset_index()

# Plot 'age' against 'year' using a line chart
plt.figure(figsize=(20, 10))
plt.plot(age_by_year['year'], age_by_year['age'], marker='o', linestyle='-', color='blue')
plt.title('Average Age vs Year')
plt.xlabel('Year')
plt.ylabel('Average Age')
plt.grid(True)
plt.show()



## Observations:
The trend in the line chart indicates the changes in the average age of individuals over the years, 
based on the receipt dates from your dataset.
Such a visualization can help identify patterns, 
such as whether the average age is increasing, decreasing, or remaining relatively stable over time.

For example:
An upward trend would suggest that the average age is increasing each year.
A downward trend would indicate that the average age is decreasing.
A flat line would imply that there is little to no change in the average age over the years.
These trends can be influenced by various factors, such as the demographics of the population being studied, 
changes in policies, or other external factors that might affect the age distribution.

In [ ]:

# Plot the Actuals

# lpa_series = lpa_df['age']
# #lpa_series = df.squeeze()
# plt.figure(figsize=(28, 14))
# plt.plot(lpa_series)
# plt.title('UK Actual LPA Data', fontsize=20)
# plt.ylabel('Age', fontsize=16)
# plt.axvline(pd.to_datetime(df['receiptdate'], errors = 'coerce').dt.year, color = 'k', linestyle='--', alpha = 0.2)
# # for year in range(min(pd.to_datetime(df['receiptdate'], errors = 'coerce').dt.year), 
# #     datetime.strptime(snapshot_end, '%Y-%m-%d').year):
# #     #datetime.strptime("2024-03-18", '%Y-%m-%d').year):
# #     plt.axvline(pd.to_datetime(df['receiptdate'], errors = 'coerce'), color = 'k', linestyle='--', alpha = 0.2)
# #     #plt.axvline(pd.to_datetime(str(year) + '-01-01'), color = 'k', 
# #     #print(year)
# plt.legend()    
# #plt.savefig('UK_Actual_LPA_Data.png', dpi=300, bbox_inches='tight')
# plt.show()  

In [ ]:
# Perform the GROUP BY operation and calculate the count
#Cases_by_year_age = lpa_df.groupby(
#    ['receiptdate', 'uid', 'type', 'casesubtype', 'status', 'donor_postcode', 'donor_gender', 'age']) \
#    .agg({'No_of_Cases': 'count'}) \ #['donor_postcode', 'donor_gender', 'age']
#    .reset_index()

#agg_funcs = dict(No_of_Cases = 'count')
#Cases_by_year_age = lpa_df.set_index(['receiptdate', 'uid', 'type', 'casesubtype', 'status', 'donor_postcode', 'donor_gender', 'age']) \
#    .stack() \
#    .groupby(level=0) \
#    .agg(agg_funcs)


#Cases_by_year_age
#lpa_by_year_age = lpa_df[['receiptdate', 'uid', 'type', 'casesubtype', 'status', 'donor_postcode', 'donor_gender', 'age']] \
#                    .groupby(['donor_postcode', 'donor_gender', 'age'])  \
#                    .agg('count')#.sum()
#lpa_by_year_age.to_csv(r'lpa_by_year_age.csv')



#lpa_df.to_csv(r'lpa_df.csv')

# Missing Data Imputation:

There are be some people in the LPA data with missing age (they are represented with negetive numbers in column age). 
So for missing data (age) imputation, his code is written to use age distribution of cases that they have age and
apply this to the total number of doners in that year. 
Actually, we allocate proportionaly distributed age across each year of these missing ages. 
E.g., if we get 90% of age distribution for a particular year,
we used this age distribution to be applied to the 100% of donors to get the total distribution. 

The code below: 
first, loads the data from the CSV file and replaces negative ages 
with NaN to represent missing data. 

It then calculates the age distribution for each year. 

For each year, it finds the indices of the missing ages and imputes 
them by randomly choosing from the age distribution of that year. 

The imputed ages are proportional to the age distribution 
of the donors that year. 

Finally, it saves the DataFrame with the imputed ages to a new CSV file.


In [ ]:
# # Function to calculate the number of unique records by age, year, gender, and postcode
# #def calculate_unique_records_by_age_year_gender_postcode(records):
# # Get the current year
# #current_year = datetime.now().year  
# #Create a dictionary to store counts for each age, year, gender, and postcode combination
# age_year_gender_postcode_counts = {}

# records = lpa_df
    
# # Iterate over each record
# for record in records:         
#     # Extract gender and postcode
#     gender = record["donor_gender"]
#     postcode = record["donor_postcode"]
#     dob = record["donor_dob"]
    
#     # Create a unique key combining age, gender, and postcode
#     key = (dob, gender, postcode)
        
#     # Increment the count for the key
#     age_year_gender_postcode_counts[key] = age_year_gender_postcode_counts.get(key, 0) + 1
        
# return age_year_gender_postcode_counts

# # Call the function and print the results
# unique_records_by_age_year_gender_postcode = calculate_unique_records_by_age_year_gender_postcode(records)

# print("Number of unique records by age, year, gender, and postcode:")

# for key, count in unique_records_by_age_year_gender_postcode.items():
#     dob, gender, postcode = key
#     print(f"Date of Birth (D.o.B): {dob}, Gender: {gender}, Postcode: {postcode}, Count: {count}")


In [ ]:
# This code will output the number of unique records for each age in each year for each donor gender in each donor postcode.
# It calculates the age based on the current year and the birth year of each person in the records.
# Then, it creates a unique key combining age, year, gender, and postcode, and increments the count for each key.
# Finally, it prints the results showing the count of unique records for each combination.

#from datetime import datetime

# Sample data representing records with donor gender, donor postcode, and date of birth
#records = [
#    {"donor_gender": "Male", "donor_postcode": "AB12 3CD", "date_of_birth": "1999-05-15"},
#    {"donor_gender": "Female", "donor_postcode": "XY34 5YZ", "date_of_birth": "1994-08-20"},
#    {"donor_gender": "Male", "donor_postcode": "CD56 7EF", "date_of_birth": "1996-02-10"},
#    {"donor_gender": "Male", "donor_postcode": "FG78 9HI", "date_of_birth": "2000-11-30"},
#    {"donor_gender": "Female", "donor_postcode": "JK90 1LM", "date_of_birth": "1987-03-25"},
#    {"donor_gender": "Male", "donor_postcode": "OP23 4QR", "date_of_birth": "1993-09-05"}
#]

# Function to calculate the number of unique records by age, year, gender, and postcode
#def calculate_unique_records_by_age_year_gender_postcode(records):
    # Get the current year
#    current_year = datetime.now().year
    
    # Create a dictionary to store counts for each age, year, gender, and postcode combination
#    age_year_gender_postcode_counts = {}
    
    # Iterate over each record
#    for record in records:
        # Extract the year of birth from the date_of_birth
#        birth_year = int(record["date_of_birth"].split("-")[0])
        
        # Calculate the age of the person
#        age = current_year - birth_year
        
        # Extract the year from the date_of_birth
#        year = birth_year
        
        # Extract gender and postcode
#        gender = record["donor_gender"]
#        postcode = record["donor_postcode"]
        
        # Create a unique key combining age, year, gender, and postcode
#        key = (age, year, gender, postcode)
        
        # Increment the count for the key
#        age_year_gender_postcode_counts[key] = age_year_gender_postcode_counts.get(key, 0) + 1
        
#    return age_year_gender_postcode_counts

# Call the function and print the results
#unique_records_by_age_year_gender_postcode = calculate_unique_records_by_age_year_gender_postcode(records)
#print("Number of unique records by age, year, gender, and postcode:")
#for key, count in unique_records_by_age_year_gender_postcode.items():
#    age, year, gender, postcode = key
#    print(f"Age: {age}, Year: {year}, Gender: {gender}, Postcode: {postcode}, Count: {count}")


# Missing age imutation

There are two issues with the age:

1. The donor_gender might be missing or entered incorrectly

2. The derieved age might be higher than 126 years old



In [ ]:
lpa_data_sample_imputed = lpa_df

# Filter rows with negative or greater than 126 age values
criteria = lpa_data_sample_imputed[(lpa_data_sample_imputed['age'] < 0) | (lpa_data_sample_imputed['age'] > 126)]

# Display the filtered rows
print(criteria)

# Replace age values with NULL (NaN) in the filtered rows
lpa_data_sample_imputed.loc[criteria.index, 'age'] = np.nan #None

# Display the updated DataFrame
print(lpa_data_sample_imputed)

# Group by year and count age groups
age_distribution = lpa_data_sample_imputed.groupby('year')['age'].value_counts()

# Fill missing ages with the most common age for each year
most_common_age = lpa_data_sample_imputed.groupby('year')['age'].apply(lambda x: x.mode().iloc[0])
lpa_data_sample_imputed['age'] = lpa_data_sample_imputed.apply(lambda row: most_common_age[row['year']] if pd.isna(row['age']) else row['age'], axis=1)

# Display the age distribution after filling missing ages
print("\nAge distribution by year (including filled missing ages):")
print(age_distribution)

# Display the final DataFrame
print("\nFinal DataFrame:")
print(lpa_data_sample_imputed)

# Save the dataframe with imputed ages
lpa_data_sample_imputed.to_csv('lpa_data_sample_imputed.csv', index=False)

# Print a success message
print("The missing age data has been successfully imputed and saved to lpa_data_sample_imputed.csv file.")

In [ ]:

# lpa_data_sample_imputed = lpa_df


# # Identify the rows with missing age (represented as negative numbers)
# ## 1. The donor_gender might be missing or entered incorrectly:  < 0
# ## 2. The derieved age might be higher than 126 years old > 126
# lpa_data_sample_imputed['missing_age'] = (lpa_data_sample_imputed['age'] < 0) | (lpa_data_sample_imputed['age'] > 126)

# # Replace negative ages with NaN
# lpa_data_sample_imputed.loc[missing_age, 'age'] = np.nan

# # Calculate the age distribution for each year excluding missing ages
# age_distribution = lpa_data_sample_imputed.loc[~missing_age].groupby('year')['age'].value_counts(normalize=True)

# # Calculate the age distribution for each year
# age_distribution_per_year = lpa_data_sample_imputed.groupby('year')['age'].value_counts(normalize=True)

# # Apply the age distribution to the total number of donors in each year
# for year in df['year'].unique():
#     # Calculate the number of missing ages in the current year
#     num_missing = missing_age & (df['year'] == year)
    
#     # If there are missing ages in the current year
#     if num_missing.sum() > 0:
#         # Generate ages according to the age distribution of the current year
#         imputed_ages = np.random.choice(age_distribution[year].index, 
#                                         p=age_distribution[year].values, 
#                                         size=num_missing.sum())
        
#         # Assign the generated ages to the missing ages
#         df.loc[num_missing, 'age'] = imputed_ages


# # Apply the age distribution to the missing ages
# for year in lpa_data_sample_imputed['year'].unique():
#     missing_age_indices = lpa_data_sample_imputed[(lpa_data_sample_imputed['year'] == year) & (lpa_data_sample_imputed['age'].isna())].index
#     if not missing_age_indices.empty:
#         imputed_ages = np.random.choice(age_distribution_per_year[year].index, 
#                                         p=age_distribution_per_year[year].values, 
#                                         size=len(missing_age_indices))
#         lpa_data_sample_imputed.loc[missing_age_indices, 'age'] = imputed_ages

      


In [ ]:
# create a unique identifier based on multiple columns:
# lpa_unique_key = lpa_df


# #df1.set_index(['donor_postcode', 'donor_gender', 'age']).index.factorize()[0]+1
# lpa_unique_key.insert(loc = 0, column='ukey', value = lpa_unique_key.set_index(['donor_postcode', 'donor_gender', 'age']).index.factorize()[0]+1)
# #lpa_unique_key

# #(lpa_unique_key.fillna({'donor_postcode':'', 'donor_gender':'', 'age':''})
# #   .groupby(['donor_postcode', 'donor_gender', 'age'],sort=False).ngroup()+1)

# #lpa_unique_key.loc[lpa_unique_key['type']=='lpa','ukey'].agg(['nunique','count','size'])
# #lpa_unique_key.query('type == "lpa"')['ukey'].agg(['nunique','count','size'])
# #lpa_unique_key.query('casesubtype == "hw"')['ukey'].agg(['nunique','count','size'])
# #lpa_unique_key.query('casesubtype == "pfa"')['ukey'].agg(['nunique','count','size'])
# #lpa_unique_key.groupby(['ukey']).count()
# #lpa_unique_key['count_ukey'] = lpa_unique_key['ukey'].value_counts()
# #lpa_unique_key



# lpa_unique_key['CountbyUkey'] = lpa_unique_key.groupby(['donor_postcode', 'donor_gender']).age.transform('count')
# lpa_unique_key['CountbyAge'] = lpa_unique_key.groupby('year').age.transform('count').sum()

# # Perform the GROUP BY operation and calculate the sum
# lpa_age = lpa_unique_key.groupby(['donor_postcode', 'donor_gender', 'age']) \
#     .agg({'CountbyAge': 'sum'}) \
#     .reset_index()

# print(lpa_age)
# #lpa_unique_key['month'] = lpa_unique_key['ArrivalDate'].dt.month


# # Cases_by_year_age

# #lpa_by_year_age = lpa_unique_key[['receiptdate', 'uid', 'type', 'casesubtype', 'status', 'donor_postcode', 'donor_gender', 'age']] \
# #                    .groupby(['donor_postcode', 'donor_gender', 'age'])  \
# #                    .agg('count')#.sum()


In [ ]:

# Number of LPA reciepts
lpa_reciepts = lpa_df
# Extract month letter and year 
###lpa_reciepts['month_year'] = lpa_reciepts['receiptdate'].dt.strftime('%b-%y')
# Calculate the number of unique records by month and year
#count_reciepts_month = lpa_reciepts.groupby(['year', 'month_year'])['receiptdate'].nunique().reset_index(name='count')
###Count_daily_reciepts = lpa_reciepts.groupby(['receiptdate']).count()

#unique_records = df.groupby('unique_key').agg('count').reset_index()  #.groupby(['year'])['unique_key'].nunique().reset_index(name='count')
###Count_daily_reciepts = Count_daily_reciepts.rename(columns={"count": "Count_of_daily_reciepts"})

# Display the result
###print(Count_daily_reciepts)

# Save the result into a csv file
lpa_reciepts.to_csv(r'lpa_reciepts.csv')

# Generate a Unique key by combining age, donor_gender, and donor_postcode

For ages over 19 years old:
Unique case reference for each donor = [donor_dob + donor_postcode + donor_gender]

In [ ]:

# DataFrame with the count of unique records for each combination of age and year. 
# Generate a unique key by combining age, donor_gender, and donor_postcode, 
# and then calculate the number of unique records by age and year.

lpa_unique = lpa_data_sample_imputed

# Remove spaces from the donor postcodes
lpa_unique['donor_postcode'] = lpa_unique['donor_postcode'].str.strip()
lpa_unique['donor_postcode'] = lpa_unique['donor_postcode'].str.replace(' ', '')

# Generate a unique key by combining age, donor_gender, and donor_postcode
lpa_unique['unique_key'] = lpa_unique['donor_dob'].astype(str) \
+ lpa_unique['donor_gender'] + lpa_unique['donor_postcode']

# lpa_by_year_age = lpa_unique_key

# lpa_by_year_age.to_csv(r'lpa_by_year_age.csv')

# remove duplicate rows based on Id values(unique_key) and 
# keep only the row that don't have 0 value in all the fields.


duplicateMask = lpa_unique.duplicated('unique_key', keep=False)

lpa_unique = pd.concat([lpa_unique.loc[duplicateMask & lpa_unique[['age', 'donor_gender', 'donor_postcode']].ne(0).any(axis=1)], \
               lpa_unique[~duplicateMask]])

#lpa_df['zero']=lpa_df.select_dtypes(['int','float']).eq(0).sum(axis=1)
#df=df.sort_values(['zero','Id']).drop_duplicates(subset=['Id']).drop(columns='zero')df['zero']=df.select_dtypes(['int','float']).eq(0).sum(axis=1)
#df=df.sort_values(['zero','Id']).drop_duplicates(subset=['Id']).drop(columns='zero')

#lpa_unique = lpa_unique.drop_duplicates(subset="unique_key")
lpa_unique

# Save the LPA data with new unique keys (as a unique ID)

Sort by the unique id

In [ ]:
# Sort the rows of dataframe by  'unique_key'  
## column inplace

#lpa_df_index = lpa_unique.sort_values(lpa_unique.columns[9])

## Sort by 'unique_key' column in ascending order
lpa_df_index = lpa_unique.sort_values(by='unique_key')


#lpa_df_index = lpa_unique.sort_values['unique_key']
#lpa_df_index = lpa_unique.sort_values(by = 'unique_key', axis = 1, inplace = True, ascending = True)
#lpa_df_index = lpa_unique.reindex(sorted(lpa_unique.columns), axis=1)

## Set index
#df['receiptdate'] = pd.to_datetime(df['receiptdate'])

#df = df.set_index('receiptdate').asfreq('D')
#lpa_df_index['unique_key'] = 

## Set the unique key as an ID (index)
lpa_df_index.set_index('unique_key', inplace = True)

#df.index = df.index.to_period('D')
                            
###print(df.head())
###print(df.tail())

#Missing_data = lpa_df_index[(lpa_data_sample_imputed['age'] < 0 | lpa_data_sample_imputed['age'] > 126)]
#print(Missing_data)



In [ ]:
# Extract and save data into a csv file
lpa_data = lpa_df_index
lpa_data.to_csv(r'lpa_data.csv')


In [ ]:
# Extract month letter and year 
lpa_data['month_year'] = lpa_data['receiptdate'].dt.strftime('%b-%y')

# Work out how many people applied for lpa and recieved the power of atthorney and how many applications in a year/month/week by age group since 2007?

In [ ]:
#count_unique_records = lpa_data.groupby('unique_key').agg('count').reset_index()
#count_unique_records

In [ ]:
d = lpa_unique.sort_values(by='unique_key')
count_unique_grouped_year = d.groupby(['year'])['unique_key'].nunique().reset_index(name='count')
count_unique_grouped_year

In [ ]:
d = lpa_unique.sort_values(by='unique_key')
count_unique_grouped_age = d.groupby(['age'])['unique_key'].nunique().reset_index(name='count')
count_unique_grouped_age

In [ ]:
g = lpa_unique
# Group by 'item' and 'color', then count the occurrences
count_unique_grouped_all = g.groupby(['receiptdate', 'uid', 'casesubtype', 'status', 'donor_dob', 'donor_postcode', 'donor_gender', 'age', 'year'])['unique_key'].count().reset_index(name='count')
count_unique_grouped_all

In [ ]:
g = lpa_unique
# Group by 'item' and 'color', then count the occurrences
count_unique_grouped_age_year = g.groupby(['age', 'year'])['unique_key'].count().reset_index(name='count')
count_unique_grouped_age_year = count_unique_grouped_age_year.rename(columns={"count": "Count_of_CASEID"})
count_unique_grouped_age_year

# Save the result into a csv file
count_unique_grouped_age_year.to_csv(r'count_unique_grouped_age_year.csv')

In [ ]:
# Extract the year of reciept from the receiptdate
#receipt_year = int(record["receiptdate"].split("-")[0])

# Calculate the number of unique records by age and year
#####count_unique_records = lpa_data.reset_index(name='count')


#unique_records = df.groupby('unique_key').agg('count').reset_index()  #.groupby(['year'])['unique_key'].nunique().reset_index(name='count')
####count_unique_records = count_unique_records.rename(columns={"count": "Count_of_CASEID"})

# Display the result
####print(count_unique_records)

# Save the result into a csv file
####count_unique_records.to_csv(r'count_unique_records.csv')

# Dermine Whether the application type [casesubtype] is hw=health and welfare or pfa=property and finance

# How many certificate provider (cp) for each lpa application?

# Location based data and geographical data for the donor can be used to identify the financial situation and wherether they are located in England or Wales

In [ ]:
# Extract the year from the receiptdate
#receipt_year = birth_year        

# Calculate the age of the person
#age = receiptdate - birth_year
#lpa_df['a'] = 
############(pd.to_datetime(lpa_df['receiptdate'], errors = 'coerce').dt.day - pd.to_datetime(lpa_df['donor_dob'], errors = 'coerce').dt.day) # / 365.25
#lpa_df

# Create an Excel writer
writer = pd.ExcelWriter('LPA_Data_actuals_Years.xlsx', engine='xlsxwriter')

# Iterate through unique years and save data to separate sheets
for year in count_unique_grouped_age_year['year'].unique():
    year_data = count_unique_grouped_age_year[count_unique_grouped_age_year['year'] == year]
    year_data.to_excel(writer, sheet_name=str(year), index=False)


# Save the Excel file
writer.save()
writer.close()  # Close the ExcelWriter

year_data
# # Iterate through unique years and save data to separate sheets
# for year in lpa_df['year'].unique():
#     year_data = lpa_df[lpa_df['year'] == year]
#     chunk_size = 100000  # Adjust as needed
#     num_chunks = len(year_data) // chunk_size + 1
#     for i in range(num_chunks):
#         start_idx = i * chunk_size
#         end_idx = (i + 1) * chunk_size
#         chunk_data = lpa_df.iloc[start_idx:end_idx]
#         chunk_data.to_excel(writer, sheet_name=f'Sheet{i}', index=False)

# # Save the Excel file
# writer.save()
# writer.close()  # Close the ExcelWriter

In [ ]:
# Pivot table with count aggregation
pivot_table = pd.pivot_table(lpa_unique,
                              values='unique_key',
                              index='age',
                              columns='year',
                              aggfunc='count')

# Replace NaN values with zeros
pivot_table_filled = pivot_table.fillna(0)

print(pivot_table_filled)

In [ ]:
# Save the result into a csv file
pivot_table_filled.to_csv(r'count_unique_records.csv')

# Define the source path of the CSV file (assuming it's in the current directory)
source_csv_path = "count_unique_records.csv"

# Define the target directory where the CSV file should be placed
target_directory = "csv_files"

# Create the target directory if it doesn't exist
if not os.path.exists(target_directory):
    os.makedirs(target_directory)

# Move the CSV file to the target directory
shutil.move(source_csv_path, os.path.join(target_directory, "count_unique_records.csv"))

# Print a success message
print(f"The CSV file {source_csv_path} was successfully moved to {target_directory}.")

# Mortality Statistics
## Source Data For Mortality Statistics and Modelled Age Specific Survival Rates (Model Input Set By Control Assumptions)

# LPA Control Assumptions
## Specific Key Assumptions that control expected demand , LPA market size and saturation.


# Meta data and Variable selection and Data Cleaning for the Mortality statastics data based on population projections:

## Goal: 
### What proportion of the UK population are likely to buy LPA and still alive?
*How many people are still alive (Living Donors bought LPA)*
*Based on ONS Data of Population of Engalnd and Wales, how many people are still alive and how many of them are dead?*
*e.g., if there are 1000 people and 100 of them are still alive and bought LPA,
so there are 900 of them still didn't buy LPA.



**1. These rates are standardised to the 2013 European Standard Population, expressed per million population; 
they allow comparisons between populations with different age structures, including between males and females and over time. 
**2.  Deaths per 1,000 live births. 
**3.  Death figures are based on deaths registered rather than deaths occurring in a calendar year.

### For information on registration delays for a range of causes, see: 
    https://webarchive.nationalarchives.gov.uk/ukgwa/20160106020016/http://www.ons.gov.uk/ons/guide-method/user-guidance/health-and-life-events/impact-of-registration-delays-on-mortality-statistics/index.html

A limiting factor in modelling numbers of surving LPA holders aged 90+ has been the absence of single age specific mortality rates 
for this group. Estimates* suggested that previously applied mortality rates were too low increasing the apparent numbers of 
surviving LPA holder saged 90+ and therefore over-estimating the "sauration of this market.

For the 2018 LPA forecast , Age specific mortality rates for those aged 90+ have therefore been extrapolated based on 
a standard log power law that best fits existing mortality rates to age. 

*numbers of surviving LPA holders were estimated to exceed the total projected  population in each age group which was 
clearly not possible.


# LPA SURVIVAL TABLES:
 LPA MODEL/LPA SURVIVAL TABLES
percentage of people are died in one year


# if a 1000 40 years old male bought an LPA in 2008, what proportion of are still alove today?

# The model taking each age categories (categorical variable) and assumed that they are 
# singe age-specifics in the age category 18 to 90 and provide figure what percentage of people for male died within one year?

## e.g., in the 15-19 age category, 0.3 percent of males died within one year in the UK and 0.03 per 1000
## e.g., in the 25-29 age category, 0.6 percent of males died within one year in the UK and 0.06 per 1000
## e.g., in the 70-74 age category, 23.7 percent of males died within one year in the UK or 2.37 per 1000

## if you started at age 18, 7 years and become 25 years old ahead, 
## as the ages goes up you will fall into a higher mortality category (from 0.3 to 0.6)




# calculate naïve extrapolation for demand forecasting and calculate low planning estimate, centeral planning estimate, and high planning estimate

In [ ]:
# Extract the month of reciept from the receiptdate
#lpa_data_no_index['month'] = lpa_data_no_index['receiptdate'].dt.month
#lpa_data_no_index

In [ ]:


# Calculate the number of unique records by month and year
count_unique_month = lpa_data.groupby(['year', 'month_year', 'age'])['uid'].nunique().reset_index(name='count')


#unique_records = df.groupby('unique_key').agg('count').reset_index()  #.groupby(['year'])['unique_key'].nunique().reset_index(name='count')
count_unique_month = count_unique_month.rename(columns={"count": "Count_of_CASEID_month"})

# Display the result
print(count_unique_month)

# Save the result into a csv file
#count_unique_month.to_csv(r'count_unique_month.csv')

In [ ]:
#df_count_unique_month = pd.DataFrame(count_unique_month)
age_lower_limit = 50
age_upper_limit = 70
count_unique_month1 = count_unique_month.loc[(count_unique_month["age"] >= age_lower_limit) &
                 (count_unique_month["age"] <= age_upper_limit)]

#age_range = [50:70]
#count_unique_month.loc[count_unique_month["month_year"].isin(age_range)]

count_unique_month2 = count_unique_month1[count_unique_month1['month_year'].isin(['Jan-24', 'Feb-24', 'Mar-24'])]
#count_unique_month2 = count_unique_month1[count_unique_month1['month_year'].isin([50:70])]
#df_count_unique_month['month_year'] = pd.to_datetime(df_count_unique_month['month_year'], format='%b-%y')
#df_count_unique_month = df_count_unique_month.sort_values(df_count_unique_month.columns[1])
#df_count_unique_month = df_count_unique_month.set_index(['month_year'])

count_unique_month_age = count_unique_month2
# Save the result into a csv file
count_unique_month_age.to_csv(r'count_unique_month_age.csv')

In [ ]:


# Calculate the naïve forecast (previous month's sales)
count_unique_month_age['Naive_Forecast'] = count_unique_month_age['Count_of_CASEID_month'].shift(1)

# Define planning estimate factors
low_factor = 0.9
high_factor = 1.1

# Calculate planning estimates
count_unique_month_age['Low_Planning_Estimate'] = count_unique_month_age['Naive_Forecast'] * low_factor
count_unique_month_age['Central_Planning_Estimate'] = count_unique_month_age['Naive_Forecast']
count_unique_month_age['High_Planning_Estimate'] = count_unique_month_age['Naive_Forecast'] * high_factor

# Calculate MAPE (Mean Absolute Percentage Error)
count_unique_month_age['Absolute_Percentage_Error'] = abs(count_unique_month_age['Count_of_CASEID_month'] - count_unique_month_age['Naive_Forecast']) / count_unique_month_age['Count_of_CASEID_month']
mape = count_unique_month_age['Absolute_Percentage_Error'].mean() * 100

# Calculate MAD (Mean Absolute Deviation)
count_unique_month_age['Absolute_Deviation'] = abs(count_unique_month_age['Count_of_CASEID_month'] - count_unique_month_age['Naive_Forecast'])
mad = count_unique_month_age['Absolute_Deviation'].mean()

# Display results
print(count_unique_month_age)
print(f"MAPE: {mape:.2f}%")
print(f"MAD: {mad:.2f}")
print("\nPlanning Estimates:")
print(f"Low Planning Estimate: {count_unique_month_age['Low_Planning_Estimate'].iloc[-1]:.2f}")
print(f"Central Planning Estimate: {count_unique_month_age['Central_Planning_Estimate'].iloc[-1]:.2f}")
print(f"High Planning Estimate: {count_unique_month_age['High_Planning_Estimate'].iloc[-1]:.2f}")

In [ ]:
# Create a line plot
plt.figure(figsize=(8, 6))
plt.plot(df_count_unique_month['Count_of_CASEID_month'], df_count_unique_month['Low_Planning_Estimate'], label='Low Estimate', marker='o')
plt.plot(df_count_unique_month['Count_of_CASEID_month'], df_count_unique_month['Central_Planning_Estimate'], label='Central Estimate', marker='s')
plt.plot(df_count_unique_month['Count_of_CASEID_month'], df_count_unique_month['High_Planning_Estimate'], label='High Estimate', marker='^')

# Add labels and title
plt.xlabel('Month')
plt.ylabel('Sales Estimate')
plt.title('Demand Forecasting Estimates')
plt.grid(True)
plt.legend()

In [ ]:

# Number of LPA reciepts
lpa_reciepts = lpa_unique
# Extract month letter and year 
lpa_reciepts['month_year'] = lpa_reciepts['receiptdate'].dt.strftime('%b-%y')
# Calculate the number of unique records by month and year
count_reciepts_month = lpa_reciepts.groupby(['year', 'month_year'])['receiptdate'].nunique().reset_index(name='count')


#unique_records = df.groupby('unique_key').agg('count').reset_index()  #.groupby(['year'])['unique_key'].nunique().reset_index(name='count')
count_reciepts_month = count_reciepts_month.rename(columns={"count": "Count_of_reciepts_month"})

# Display the result
print(count_reciepts_month)

# Save the result into a csv file
count_reciepts_month.to_csv(r'count_reciepts_month.csv')